In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy.fftpack as fft
from astropy.io import fits
from scipy import optimize
import poppy
import gpipsfs
from scipy import signal
import os
import pytz
import time
import re

Kipac paths

In [ ]:
file1 =  '/Users/MelisaT/Documents/Research/GPIDomeSeeing/data/Reduced/20160229/aored_When_2016.2.29_5.33.19_poldm_phase.fits'
file2 =  '/Users/MelisaT/Documents/Research/GPIDomeSeeing/data/Reduced/20160227/aored_When_2016.2.27_0.2.8_poldm_phase.fits'

laptop paths

In [ ]:
file1 =  '/Users/melisatallis/Documents/Research/GPIDomeSeeing/data/aotelem/aored_When_2016.2.27_0.40.14_poldm_phase.fits'
file2 =  '/Users/melisatallis/Documents/Research/GPIDomeSeeing/data/aotelem/aored_When_2016.2.29_22.52.44_poldm_phase.fits'

Import images & remove static aberrations

In [ ]:
def import_fits(filepath):

    hdulist = fits.open(filepath,memmap=True)
    phase = hdulist[0].data.astype('float')

    return phase

def remove_zernikes(phase):
    
    m1 = gpipsfs.GeminiPrimary().sample(npix=48)
    avg_phase = np.mean(phase*m1,axis=0)
    
    z_basis = poppy.zernike.zernike_basis_faster(nterms= 6, npix = 48)
    z_coeff = poppy.zernike.opd_expand_nonorthonormal(avg_phase, aperture=m1, nterms=6)
    thin_lens = np.sum(z_coeff[:,None,None]*z_basis[:,:,:],axis=0)
    c_phase = (phase - thin_lens[None,:,:])*m1
    c_phase[np.isnan(c_phase)]=0.

    return c_phase

In [ ]:
phase = import_fits(file1)
c_phase = remove_zernikes(phase)
img = c_phase[0,:,:]

Indexing 

In [ ]:
%matplotlib notebook
plt.imshow(r)

In [ ]:
df = pd.DataFrame(columns=['r','D'])

r = []
D = []
for m in np.arange(15,25):
    for n in np.arange(15,25):
        print(m,n)
        print(np.hypot(m,n))
        r.append(np.hypot(m,n))
        D.append((img[m,n] - img[15,15])**2)       
        
df['r'] = r
df['D'] = D

In [ ]:
df.hist()